<a href="https://colab.research.google.com/github/Varun9213/CE-903-DEC/blob/main/Demo_DEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple Clustering==0.0.13 >> /dev/null

In [4]:
from Clustering.Clustering import DEC

In [5]:
# import numpy as np
# import pandas as pd

# import tensorflow as tf
# import tensorflow.keras.backend as K
# import tensorflow.keras.layers as L
from tensorflow.keras.optimizers import SGD, Adam
# from tensorflow.keras.models import Model


from sklearn.datasets import load_iris, load_digits
from sklearn.cluster import KMeans
from sklearn import metrics
import numpy as np
import pandas as pd

In [6]:
fea, tar = load_digits(return_X_y=True)
fea = np.array(fea)
tar = np.array(tar)

In [7]:
%mkdir iris_DEC

In [8]:
dec = DEC([fea.shape[1],128,64,64,32,32],nclusters=10)
dec.autoencoder.summary()

Model: "Autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ (InputLayer)          [(None, 64)]              0         
_________________________________________________________________
encoding_layer_0 (Dense)     (None, 128)               8320      
_________________________________________________________________
encoding_layer_1 (Dense)     (None, 64)                8256      
_________________________________________________________________
encoding_layer_2 (Dense)     (None, 64)                4160      
_________________________________________________________________
encoding_layer_3 (Dense)     (None, 32)                2080      
_________________________________________________________________
encoder_output (Dense)       (None, 32)                1056      
_________________________________________________________________
decoding_layer_4 (Dense)     (None, 32)                

In [9]:
dec.train_autoencoder(fea,epochs=5000,dir="/content/iris_DEC",optimizer=Adam(learning_rate=0.001))

loss : 0.78802
Weights saved to /content/iris_DEC/autoencoder_weights.h5


In [10]:
dec.compile(optimizer=Adam(learning_rate=0.001), loss='kld')
dec.fit(fea, tar, epochs=1000, dir="/content/iris_DEC")

Initializing cluster centers with k-means.........
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ (InputLayer)          [(None, 64)]              0         
_________________________________________________________________
encoding_layer_0 (Dense)     (None, 128)               8320      
_________________________________________________________________
encoding_layer_1 (Dense)     (None, 64)                8256      
_________________________________________________________________
encoding_layer_2 (Dense)     (None, 64)                4160      
_________________________________________________________________
encoding_layer_3 (Dense)     (None, 32)                2080      
_________________________________________________________________
encoder_output (Dense)       (None, 32)                1056      
_________________________________________________________________
cluster_la

In [11]:
y_pred = dec.predict(fea)

In [12]:
y_pred

array([4, 3, 3, ..., 3, 5, 5])

In [13]:
cm = metrics.cluster.contingency_matrix(tar,y_pred)
cm

array([[  0,   0,   0,   0, 177,   0,   0,   0,   0,   1],
       [  0,  55,   0,  99,   0,   0,  27,   1,   0,   0],
       [  0,   0,   1,   9,   0,   0, 167,   0,   0,   0],
       [  0,   0,   1,  10,   0,  13,   2,   1, 156,   0],
       [  0,   0,   4,   3,   0,   0,   0,   0,   0, 174],
       [  0,   0,   0,   0,   0,  52,   0, 130,   0,   0],
       [179,   0,   0,   2,   0,   0,   0,   0,   0,   0],
       [  0,   0, 178,   1,   0,   0,   0,   0,   0,   0],
       [  0,   3,   0,  91,   0,  79,   1,   0,   0,   0],
       [  0,  21,   9,   2,   1, 145,   0,   2,   0,   0]])

In [14]:
def acc_clustering(cm,y):
  df = pd.DataFrame(cm)
  true = 0
  max_val = []
  for col in df:
    sorted = df[col].sort_values(ascending = False)
    i = 0
    while sorted.index[i] in max_val:
      i += 1
    true += sorted.values[i]
    max_val.append(sorted.index[i])

  return (true/len(y))*100


In [15]:
acc_clustering(cm,tar)

80.80133555926544

In [16]:
y_pred

array([4, 3, 3, ..., 3, 5, 5])

In [17]:
tar

array([0, 1, 2, ..., 8, 9, 8])